In [1]:
import pandas as pd
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
#from keras.callbacks import EarlyStopping
from sklearn import preprocessing
import matplotlib.pyplot as plt
#import tensorflow_probability as tfp
#from tensorflow_probability import edward2 as ed
import tensorflow as tf
import edward as ed

ModuleNotFoundError: No module named 'edward'

In [3]:
!pip install edward==1.3.0

     |████████████████████████████████| 51kB 1.2MB/s eta 0:00:011
  Created wheel for edward: filename=edward-1.3.0-cp37-none-any.whl size=66977 sha256=2ba075eb156dd9824bf7e25a4cf903e2b49fc09c912d304d0d362ae60c745202
  Stored in directory: /Users/baronma/Library/Caches/pip/wheels/2b/9e/2f/671e24e430451bed7ffd55b9182a4889f25d76e5c2e212f2b4
Successfully built edward
  Found existing installation: edward 1.3.5
    Uninstalling edward-1.3.5:
      Successfully uninstalled edward-1.3.5
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data and do data normalization

In [3]:
df = pd.read_csv('AAPL.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,The Date of this Price,Expiration Date of the Option,Strike Price,Lowest Closing Ask Across All Exchanges,Implied Volatility of the Option,Underlying Price,DGS10,maturity,"C=Call, P=Put_C","C=Call, P=Put_P"
0,0,2012-09-04,2012-09-07,590.0,85.50,0.636391,674.97,1.59,3,1,0
1,4,2012-09-04,2012-09-07,610.0,65.45,0.495483,674.97,1.59,3,1,0
2,5,2012-09-04,2012-09-07,615.0,60.25,0.408294,674.97,1.59,3,1,0
3,6,2012-09-04,2012-09-07,620.0,55.25,0.376411,674.97,1.59,3,1,0
4,7,2012-09-04,2012-09-07,625.0,50.25,0.363375,674.97,1.59,3,1,0


In [4]:
df.drop(['Unnamed: 0', 'The Date of this Price', 'Expiration Date of the Option'], axis = 1, inplace = True)
df.rename(columns = {'Implied Volatility of the Option': 'IV', 'DGS10':'Risk-free interest rate', 'maturity':'Maturity'}, inplace=True)
df.columns.values[1] = "Price"
df.head()

,Strike Price,Price,IV,Underlying Price,Risk-free interest rate,Maturity,"C=Call, P=Put_C","C=Call, P=Put_P"
0,590.0,85.50,0.636391,674.97,1.59,3,1,0
1,610.0,65.45,0.495483,674.97,1.59,3,1,0
2,615.0,60.25,0.408294,674.97,1.59,3,1,0
3,620.0,55.25,0.376411,674.97,1.59,3,1,0
4,625.0,50.25,0.363375,674.97,1.59,3,1,0


In [5]:
df_sample = df.sample(n=100000)

In [6]:
target = df_sample['Price'].values
data = df_sample[['Strike Price', 'IV', 'Underlying Price', 'Risk-free interest rate', 'Maturity', 'C=Call, P=Put_C', 'C=Call, P=Put_P']]
#df.drop(['Price'], axis = 1, inplace = True)

In [7]:
data['Risk-free interest rate']=data['Risk-free interest rate'].convert_objects(convert_numeric=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

In [9]:
print(X_train.shape)
print(y_train.shape)

(67000, 7)
(67000,)


In [10]:
n_train = X_train.shape[0]

## Bayesian Neural Network Model

We define a neural network with two hidden layers of dimensions $H_0$ and $H_1$. With $D$-dimensional input.

In [11]:
D = X_train.shape[1]
H0 = 10
H1 = 20

def neural_network(X,W_0,W_1,W_2,b_0,b_1,b_2):
    hidden1 = tf.nn.relu(tf.matmul(X,W_0) + b_0)
    hidden2 = tf.nn.relu(tf.matmul(hidden1,W_1) + b_1)
    output = tf.matmul(hidden2,W_2) + b_2
    return tf.reshape(output, [-1])

Let us denote the parameters of the neural network by $\theta = \{ \mathbf{W}_0, \mathbf{W}_1, \mathbf{W}_2, \mathbf{b}_0, \mathbf{b}_1, b_2 \}$, and define the prior $p(\theta)$ by setting each component of each parameter to have a standard normal distribution. We then define the likelihood as a normal distribution with fixed variance $\sigma_y^2$:

In [16]:
tfd = tfp.distributions

In [14]:
scaler = preprocessing.StandardScaler().fit(X_train)

data_train_scaled = scaler.transform(X_train)
data_test_scaled = scaler.transform(X_test)

sigma_y = 1.0

from edward.models import Normal, NormalWithSoftplusScale

W_0 = Normal(loc=tf.zeros([D, H0]), scale=5.0 * tf.ones([D, H0]))
W_1 = Normal(loc=tf.zeros([H0, H1]), scale=5.0 * tf.ones([H0, H1]))
W_2 = Normal(loc=tf.zeros([H1, 1]), scale=5.0 * tf.ones([H1, 1]))
b_0 = Normal(loc=tf.zeros(H0), scale=5.0 * tf.ones(H0))
b_1 = Normal(loc=tf.zeros(H1), scale=5.0 * tf.ones(H1))
b_2 = Normal(loc=tf.zeros(1), scale=5.0 * tf.ones(1))


#tf.compat.v1.disable_eager_execution()
X = tf.placeholder(tf.float32, [None, D])
y = Normal(loc=neural_network(X,W_0,W_1,W_2,b_0,b_1,b_2), scale=sigma_y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


Since exact inference of the posterior is intractable, we resort to methods from variational inference. This amounts to approximating the posterior $p( \, \theta \, | \, \mathcal{D} \, )$ by a parametrized family $q( \, \theta \, ; \, \lambda \, )$, which we choose to consist of normal variables of unknown mean and variance (represented here by $\lambda$).

In [15]:
qW_0 = NormalWithSoftplusScale(loc=tf.Variable(tf.random_normal([D, H0])), 
                               scale=tf.Variable(tf.random_normal([D, H0])))
qW_1 = NormalWithSoftplusScale(loc=tf.Variable(tf.random_normal([H0, H1])),
                               scale=tf.Variable(tf.random_normal([H0, H1])))
qW_2 = NormalWithSoftplusScale(loc=tf.Variable(tf.random_normal([H1, 1])), 
                               scale=tf.Variable(tf.random_normal([H1, 1])))
qb_0 = NormalWithSoftplusScale(loc=tf.Variable(tf.random_normal([H0])), 
                               scale=tf.Variable(tf.random_normal([H0])))
qb_1 = NormalWithSoftplusScale(loc=tf.Variable(tf.random_normal([H1])),
                               scale=tf.Variable(tf.random_normal([H1])))
qb_2 = NormalWithSoftplusScale(loc=tf.Variable(tf.random_normal([1])),
                               scale=tf.Variable(tf.random_normal([1])))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tfd.Normal(loc, tf.nn.softplus(scale)) instead.


We choose $\lambda^*$ by minimizing the Kullback-Leibler divergence (from $q$ to $p$):

In [21]:
inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                     W_1: qW_1, b_1: qb_1,
                     W_2: qW_2, b_2: qb_2}, data={X: data_train_scaled, y: y_train})

#global_step = tf.Variable(0, trainable=False)
#starter_learning_rate = 0.05
#learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           #10000, 0.3, staircase=True)

optimizer = tf.train.AdamOptimizer(0.1)

inference.run(n_iter=2000, optimizer=optimizer)

ValueError: cannot add an op with id 2064 as it already exists in the graph

## Reference: 
&emsp;&emsp;What does 'Accuracy' mean in Regression? https://github.com/keras-team/keras/issues/7947 <br>
&emsp;&emsp;https://keras.io/metrics/<br>
&emsp;&emsp;Scale, Standardize, or Normalize with Scikit-Learn https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02 <br>
&emsp;&emsp;The Day my Computer Won the Nobel Prize (Neural Network Option Pricing)  https://medium.com/datadriveninvestor/the-day-my-computer-won-the-nobel-prize-neural-network-option-pricing-d29b4379f1d2 <br>